In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets, utils
from torch.utils import data

In [13]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")
print(USE_CUDA)

True


In [14]:
epochs = 30
batch_size = 64

In [15]:
transform = transforms.Compose([
    transforms.ToTensor()
])

trainset = datasets.FashionMNIST(
    root='./data/',
    train=True,
    download=True,
    transform=transform
)

testset = datasets.FashionMNIST(
    root='./data/',
    train=False,
    download=True,
    transform=transform
)

train_loader = data.DataLoader(
    dataset=trainset,
    batch_size=batch_size,
    shuffle=True
)

test_loader = data.DataLoader(
    dataset=testset,
    batch_size=batch_size,
    shuffle=True
)

In [22]:
class Net(nn.Module):
    def __init__(self, dropout_p=0.2):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 256)
        self.fc2 = nn.Linear(256, 64)
        self.fc3 = nn.Linear(64, 10)
        self.dropout_p = dropout_p

    def forward(self, x):
        x = x.view(-1, 784)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [26]:
model = Net(dropout_p=0.2).to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [27]:
def train(model, train_loader, optimizer):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()

In [28]:
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0

    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)

            test_loss += F.cross_entropy(output,
                                         target, reduction='sum').item()

            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()

        test_loss /= len(test_loader.dataset)
        test_accuracy = 100. * correct / len(test_loader.dataset)
        return test_loss, test_accuracy

In [29]:
for epoch in range(1, epochs + 1):
    train(model, train_loader, optimizer)
    test_loss, test_accuracy = evaluate(model, test_loader)

    print(epoch, test_loss, test_accuracy)

1 0.8508938646316528 67.13
2 0.6820990683555603 76.55
3 0.5838669384002686 79.64
4 0.544258696937561 80.92
5 0.5276677844047546 81.04
6 0.510129658985138 82.18
7 0.5050875476360321 82.06
8 0.4733560626983643 83.33
9 0.47680047159194944 83.12
10 0.5002312390327454 81.34
11 0.45228082387447355 83.95
12 0.4394407302379608 84.4
13 0.44208114452362063 84.7
14 0.4359129050731659 84.77
15 0.4435635085105896 84.47
16 0.41315384035110475 85.4
17 0.43446242933273316 84.89
18 0.4105777485847473 85.56
19 0.4108646228790283 85.43
20 0.3980354763031006 85.85
21 0.39479244871139524 86.11
22 0.41390354437828064 85.53
23 0.3917588299274445 86.2
24 0.39933037996292114 85.68
25 0.38730060834884644 86.13
26 0.3809623489379883 86.62
27 0.3810816942691803 86.23
28 0.3806058372497559 86.45
29 0.3673354778766632 87.1
30 0.371818412399292 86.69
